### Обрабатываем и обучаемся на train

In [2]:
import pandas as pd
my_csv = pd.read_csv('/Users/alinashaymardanova/Desktop/ruscorpora_parsed.csv', sep='/')

In [3]:
my_csv.head()

,Словоформа,Лемма,Морфологический разбор
0,Якутское,якутский,"A=n,sg,nom,plen"
1,отделение,отделение,"S,n,inan=sg,nom"
2,Единой,единый,"A=f,sg,gen,plen"
3,России,Россия,"S,f,inan=sg,gen"
4,планирует,планировать,"V,ipf,tran=sg,act,praes,3p,indic"


In [4]:
import re
POS = []
for my_data in my_csv['Морфологический разбор']:
    res = re.search('[A-Z]*(-[A-Z]*)?', my_data)
    POS.append(res.group(0))
my_csv['POS'] = POS
    

In [5]:
my_csv.head()

,Словоформа,Лемма,Морфологический разбор,POS
0,Якутское,якутский,"A=n,sg,nom,plen",A
1,отделение,отделение,"S,n,inan=sg,nom",S
2,Единой,единый,"A=f,sg,gen,plen",A
3,России,Россия,"S,f,inan=sg,gen",S
4,планирует,планировать,"V,ipf,tran=sg,act,praes,3p,indic",V


In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
real_1 = pd.Series(label_encoder.fit_transform(my_csv['POS'].astype(str)))
dict1 = dict(enumerate(label_encoder.classes_))

### Кодируем train data

In [7]:
categorical_columns = my_csv.columns[my_csv.dtypes == 'object'].union(['POS'])
for column in categorical_columns:
    my_csv[column] = label_encoder.fit_transform(my_csv[column].astype(str))
my_csv.head()

,Словоформа,Лемма,Морфологический разбор,POS
0,29302,50918,523,1
1,82438,32539,1328,17
2,11064,19704,453,1
3,22805,9577,941,17
4,87665,34594,2404,19


In [8]:
y = my_csv['POS']
my_csv = my_csv.drop('POS', 1)

In [9]:
my_csv = my_csv.drop('Морфологический разбор', 1)
my_csv.head()

,Словоформа,Лемма
0,29302,50918
1,82438,32539
2,11064,19704
3,22805,9577
4,87665,34594


In [10]:
onehot_encoder = OneHotEncoder(sparse=True)
X = onehot_encoder.fit_transform(my_csv)

### Мы обучаемся на train методом LogisticRegression

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
clf = LogisticRegression(n_jobs=-1, verbose=5)
clf = clf.fit(X, y)

/Users/alinashaymardanova/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


[LibLinear]

### Открываем текст test и золотой стандарт, проводим препроцессинг

Мы токенизируем текст и предстказываем части речи токенов по средствам обученного алгоритма. После этого сравниваем с золотым стандартом. Но что-то пошло не так, потому что золотой стандарт не совпадает с текстом. Поэтому, далее мы решили применить алгоритм к леммам из золотого стандарта, расставить тэги и сравить с резульататми золотого стандарта.

In [12]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
import matplotlib.pyplot as plt
%matplotlib inline 

morph = MorphAnalyzer()

with open ('/Users/alinashaymardanova/Downloads/TestSet/test.txt', 'r', encoding = 'Windows-1251') as f:
    text = f.read()

def first_normalize(text):
    lemmas = []
    text = text.lower()
    text = re.sub('!|\#|\$|%|\&|\(|\)|\*|\+|,|-|\.|\/|\:|;|<|=|>|\?|\@|\[|\]|^|_|`|\{|\}|~|«|«|»|"|…', '', text)
    text = re.sub('(http://|https://|www.)[a-zA-z0-9]*? ', '', text)
    text = re.sub('[A-Za-z0-9]+', '', text)
    text = re.sub('\s+|\n|\t', ' ', text)
    tokens = wordpunct_tokenize(text)
    for el in tokens:
        lemmas.append(morph.parse(el)[0].normal_form)
    
    clean_lemmas = [lemma for lemma in lemmas]
    
    return tokens, clean_lemmas

tokens, clean_lemmas = first_normalize(text)

new_data = pd.DataFrame()
new_data['Словоформа'] = tokens
new_data['Лемма'] = clean_lemmas

In [13]:
real_11 = pd.Series(label_encoder.fit_transform(new_data['Словоформа'].astype(str)))
categorical_columns = new_data.columns[new_data.dtypes == 'object'].union(['Словоформа'])
for column in categorical_columns:
    new_data[column] = label_encoder.fit_transform(new_data[column].astype(str))
new_data.head()


,Словоформа,Лемма
0,1504,1243
1,289,231
2,1489,1227
3,794,645
4,1244,1025


In [14]:
X1 = onehot_encoder.transform(new_data)
X1

<2322x184026 sparse matrix of type '<class 'numpy.float64'>'
	with 4644 stored elements in Compressed Sparse Row format>

In [15]:
Y_pred = clf.predict(X1)

In [16]:
Y_pred

array([11, 17, 17, ..., 17, 17, 17])

In [17]:
with open ('/Users/alinashaymardanova/Downloads/GoldStandard.txt', 'r', encoding='utf-8') as f:
    gold_text = f.read()

In [18]:
gold_text = re.sub('!|\#|\$|%|\&|\(|\)|\*|\+|,|-|\.|\/|\:|;|<|=|>|\?|\@|\[|\]|^|_|`|\{|\}|~|«|«|»|"|…', '', gold_text)
gold_text = re.sub('[a-z0-9]+', '', gold_text)
gold_text = re.sub('\n+', '\n', gold_text)

In [153]:
# with open ('GOLD.txt', 'w', encoding = 'utf-8') as f:
#     f.write(gold_text)

In [18]:
new_csv = pd.read_csv('/Users/alinashaymardanova/Downloads/gold.txt', encoding = 'Windows-1251', sep=' ')
new_csv = new_csv.drop('Unnamed: 3', 1)
new_csv = new_csv.dropna()
new_csv.columns = ['Word', 'Lemma', 'POS']

In [20]:
new_csv.head()

,Word,Lemma,POS
0,Цветки,цветок,S
1,дикорастущих,дикорастущий,A
2,форм,форма,S
3,обыкновенной,обыкновенный,A
4,сирени,сирень,S


## А ТЕПЕРЬ НОРМАЛЬНО

In [21]:
real_11 = pd.Series(label_encoder.fit_transform(new_csv['POS'].astype(str)))
dict2 = dict(enumerate(label_encoder.classes_))
categorical_columns = new_csv.columns[new_csv.dtypes == 'object'].union(['POS'])
for column in categorical_columns:
    new_csv[column] = label_encoder.fit_transform(new_csv[column].astype(str))

new_csv.head()

,Word,Lemma,POS
0,298,1379,7
1,577,356,0
2,1739,1364,7
3,1042,761,0
4,1504,1159,7


In [22]:
y_test = new_csv['POS']
new_csv = new_csv.drop('POS', 1)

In [23]:
X2 = onehot_encoder.transform(new_csv)
X2

<2661x184026 sparse matrix of type '<class 'numpy.float64'>'
	with 5322 stored elements in Compressed Sparse Row format>

In [24]:
Y_pred = clf.predict(X2)

In [25]:
Y_pred

array([17, 11, 17, ..., 17, 17, 17])

In [26]:
Y_pr = Y_pred.tolist()
print(len(Y_pr))
Y_ts = y_test.tolist()
print(len(Y_ts))

2661
2661


In [27]:
Y_pr_n = []
for y in Y_pr:
    for key, value in dict1.items():
        if y == key:
            y = dict1[key]
            Y_pr_n.append(y)
Y_pr_n


['S',
 'NUM',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'NUM',
 'S',
 'NUM',
 'NUM',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'ANUM',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'NONLEX',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'NUM',
 'S',
 'S',
 'S',
 'NUM',
 'NUM',
 'NUM',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'ANUM',
 'S',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'NUM',
 'NUM',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'NUM',
 'S',
 'NUM',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'S',
 'NUM',
 'S',
 'NUM',
 'NUM',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'NUM',
 'S',
 'NUM',
 'S',
 'S',
 'NUM',

In [28]:
Y_ts_n = []
for y in Y_ts:
    for key, value in dict2.items():
        if y == key:
            y = dict2[key]
            Y_ts_n.append(y)
Y_ts_n

['S',
 'A',
 'S',
 'A',
 'S',
 'A',
 'V',
 'A',
 'S',
 'PR',
 'S',
 'PR',
 'S',
 'A',
 'S',
 'PR',
 'S',
 'PR',
 'S',
 'S',
 'S',
 'S',
 'V',
 'A',
 'PR',
 'S',
 'S',
 'A',
 'S',
 'PR',
 'S',
 'S',
 'PR',
 'S',
 'S',
 'CONJ',
 'S',
 'V',
 'A',
 'S',
 'A',
 'S',
 'S',
 'ADV',
 'V',
 'PR',
 'A',
 'S',
 'PR',
 'ADV',
 'A',
 'S',
 'PR',
 'A',
 'S',
 'PR',
 'A',
 'S',
 'PR',
 'A',
 'S',
 'V',
 'S',
 'S',
 'ADV',
 'V',
 'ADV',
 'S',
 'S',
 'PR',
 'S',
 'A',
 'S',
 'CONJ',
 'V',
 'A',
 'S',
 'ADV',
 'S',
 'S',
 'S',
 'ADV',
 'V',
 'S',
 'A',
 'S',
 'PR',
 'S',
 'S',
 'PR',
 'S',
 'S',
 'PR',
 'A',
 'A',
 'S',
 'A',
 'S',
 'V',
 'PR',
 'S',
 'S',
 'CONJ',
 'V',
 'PR',
 'S',
 'PR',
 'S',
 'S',
 'S',
 'ADV',
 'PR',
 'A',
 'A',
 'ADV',
 'S',
 'V',
 'PR',
 'S',
 'PR',
 'S',
 'S',
 'S',
 'ADV',
 'S',
 'A',
 'CONJ',
 'A',
 'S',
 'PR',
 'APRO',
 'S',
 'SPRO',
 'ADV',
 'APRO',
 'S',
 'V',
 'PR',
 'S',
 'S',
 'ADV',
 'SPRO',
 'V',
 'SPRO',
 'PR',
 'S',
 'S',
 'ADV',
 'ADV',
 'SPRO',
 'V',
 'A',
 'CONJ'

In [29]:
Y_ts = []
for y in Y_ts_n:
    for key, value in dict1.items():
        if y == dict1[key]:
            y = key
        if (y == 'APRO') or (y == 'SPRO'):
            y = 20
    Y_ts.append(int(y))
Y_ts


[17,
 1,
 17,
 1,
 17,
 1,
 19,
 1,
 17,
 14,
 17,
 14,
 17,
 1,
 17,
 14,
 17,
 14,
 17,
 17,
 17,
 17,
 19,
 1,
 14,
 17,
 17,
 1,
 17,
 14,
 17,
 17,
 14,
 17,
 17,
 7,
 17,
 19,
 1,
 17,
 1,
 17,
 17,
 4,
 19,
 14,
 1,
 17,
 14,
 4,
 1,
 17,
 14,
 1,
 17,
 14,
 1,
 17,
 14,
 1,
 17,
 19,
 17,
 17,
 4,
 19,
 4,
 17,
 17,
 14,
 17,
 1,
 17,
 7,
 19,
 1,
 17,
 4,
 17,
 17,
 17,
 4,
 19,
 17,
 1,
 17,
 14,
 17,
 17,
 14,
 17,
 17,
 14,
 1,
 1,
 17,
 1,
 17,
 19,
 14,
 17,
 17,
 7,
 19,
 14,
 17,
 14,
 17,
 17,
 17,
 4,
 14,
 1,
 1,
 4,
 17,
 19,
 14,
 17,
 14,
 17,
 17,
 17,
 4,
 17,
 1,
 7,
 1,
 17,
 14,
 20,
 17,
 20,
 4,
 20,
 17,
 19,
 14,
 17,
 17,
 4,
 20,
 19,
 20,
 14,
 17,
 17,
 4,
 4,
 20,
 19,
 1,
 7,
 1,
 17,
 17,
 19,
 19,
 17,
 1,
 17,
 7,
 19,
 1,
 17,
 1,
 17,
 17,
 4,
 7,
 4,
 19,
 14,
 19,
 17,
 4,
 19,
 4,
 1,
 17,
 20,
 19,
 14,
 17,
 4,
 7,
 1,
 19,
 19,
 17,
 17,
 14,
 17,
 20,
 17,
 14,
 20,
 19,
 14,
 17,
 14,
 17,
 14,
 1,
 17,
 17,
 20,
 1,
 17,
 4,
 19,
 14,


In [30]:
import numpy as np

Y_ts = np.asarray(Y_ts)
Y_ts


array([17,  1, 17, ..., 14, 17, 17])

In [31]:
from sklearn.metrics import accuracy_score

print(accuracy_score(Y_ts, Y_pred))

0.317925591883


In [33]:
from sklearn.metrics import classification_report

print(classification_report(Y_ts, Y_pred))

             precision    recall  f1-score   support

          1       0.00      0.00      0.00       299
          4       0.00      0.00      0.00       197
          6       0.00      0.00      0.00         8
          7       0.00      0.00      0.00       148
         10       0.00      0.00      0.00         0
         11       0.01      0.21      0.01        14
         14       0.00      0.00      0.00       352
         17       0.42      0.76      0.54      1103
         19       0.00      0.00      0.00       376
         20       0.00      0.00      0.00       164

avg / total       0.17      0.32      0.22      2661



/Users/alinashaymardanova/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/alinashaymardanova/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, n_jobs=-1, verbose=5) 
clf = clf.fit(X, y)


building tree 4 of 200building tree 2 of 200building tree 3 of 200
building tree 1 of 200


building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 44.5min


building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 607.5min


building tree 69 of 200
building tree 70 of 200
building tree 71 of 200
building tree 72 of 200
building tree 73 of 200
building tree 74 of 200
building tree 75 of 200
building tree 76 of 200
building tree 77 of 200
building tree 78 of 200
building tree 79 of 200
building tree 80 of 200
building tree 81 of 200
building tree 82 of 200
building tree 83 of 200
building tree 84 of 200
building tree 85 of 200
building tree 86 of 200
building tree 87 of 200
building tree 88 of 200
building tree 89 of 200
building tree 90 of 200
building tree 91 of 200
building tree 92 of 200
building tree 93 of 200
building tree 94 of 200
building tree 95 of 200
building tree 96 of 200
building tree 97 of 200
building tree 98 of 200
building tree 99 of 200
building tree 100 of 200
building tree 101 of 200
building tree 102 of 200
building tree 103 of 200
building tree 104 of 200
